# Homework 2

# Set up

## Installing packages

In [28]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [29]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('GEMINI_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [30]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 9.97MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 31.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 9.75MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 19.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 23.8MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [31]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
John Smith

Marketing Professional

+ Singapore, Singapore (cid:209) Kowloon

Experience

Engineer, ByteDance

2020 – Present

• Worked in a fast-paced, global technology environment.

• Collaborated across teams to support large-scale platforms.

• Applied analytical and problem-solving skills in production systems.

Education

McGill University

Bachelor of Science (BSc) in Marketing

Skills

Content Creation

SEO

Social Media

Graduated 2009

1





📄 CV_2.pdf
Minh Pham
Design Professional

Beijing, China | Hong Kong

Professional Experience

Manager, BCG

2022 – Present

• Led cross-functional teams on client-facing design initiatives.

• Managed project timelines, deliverables, and stakeholder communication.

• Applied design thinking to business and strategy problems.

Analyst, Tencent

2013 – 2017

• Conducted market and product analysis to support decision-making.

• Collaborated with design and engineering teams.

• Produced reports and insights for senior leadersh

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [32]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [33]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_VERTEX_API_KEY,
    temperature=0,
)

# from langchain_openai import ChatOpenAI
# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
# llm = ChatOpenAI(
#     model="deepseek-chat",          # or "deepseek-reasoner"
#     api_key=DEEPSEEK_API_KEY,
#     base_url="https://api.deepseek.com/v1",
#     temperature=0,
# )

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c4793-4d8c-7380-a984-16a6a250998b-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'ba9b29de-c84d-4a6c-94d7-fb295a7e4884', 'type': 'tool_call'}, {'name': 'search_facebook_users', 'args': {'q': 'Alice'}, 'id': '8958476d-4fe1-4b36-9e10-106aa19493d2', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2568, 'output_tokens': 13, 'total_tokens': 2581, 'input_token_details': {'cache_read': 0}}


In [34]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
# await tools[5].ainvoke({'person_id': 456})

# Agent Code

In [39]:
import json
import time
from typing import List, Dict, Any, Optional
from langchain_core.messages import SystemMessage, HumanMessage

########################################################
# Agent modules
########################################################

# Module 1: Convert CV text to standardized template
def convert_cv_to_standardized_template(cv_text: str, llm) -> dict:
    """Convert raw CV text to standardized template format."""

    template_example = {
        "name": "Billy Wang",
        "headline": "Investment Professional",
        "city": "Central",
        "country": "Hong Kong",
        "skills": ["Pytorch", "Investing", "Accounting"],
        "experience": [
            {
                "company": "HSBC",
                "title": "Investor",
                "seniority": "senior",
                "start_year": 2020,
                "end_year": None
            }
        ],
        "education": [
            {
                "school": "Oxford University",
                "degree": "BSc",
                "field": "Economics",
                "end_year": 2009
            }
        ]
    }

    prompt = f"""You are a CV parser. Your task is to extract information from a CV text and convert it into a standardized JSON format.

IMPORTANT: You MUST strictly follow the template structure below. This template is similar to LinkedIn format but with some differences:

DIFFERENCES from LinkedIn:
1. Skills are a simple list of strings (not objects with proficiency)
2. Experience does NOT include "is_current" field
3. Education does NOT include "start_year" field
4. We do NOT need: industry, status, years_experience, summary

REQUIRED TEMPLATE STRUCTURE:
{{
    "name": "Full name",
    "headline": "Professional headline/title",
    "city": "city",
    "country": "country",
    "skills": ["skill1", "skill2", "skill3"],
    "experience": [
        {{
            "company": "Company name",
            "title": "Job title",
            "seniority": "junior|mid|senior",
            "start_year": 2020,
            "end_year": null
        }}
    ],
    "education": [
        {{
            "school": "University name",
            "degree": "BSc|MSc|MBA|PhD",
            "field": "Field of study",
            "end_year": 2009
        }}
    ]
}}

EXAMPLE:
{json.dumps(template_example, indent=2, ensure_ascii=False)}

INSTRUCTIONS:
1. Extract all information from the CV text below
2. If information is missing, use null for fields (except skills which should be empty list [])
3. For seniority: infer from job title (e.g., "Senior Engineer" → "senior", "Analyst" → "junior" or "mid")
4. For end_year in experience: use null if the position is current/ongoing
5. Return ONLY valid JSON, no additional text or explanation
6. Ensure all years are integers (not strings)

CV TEXT TO PARSE:
{cv_text}

Return the standardized JSON structure:"""

    messages = [
        SystemMessage(content="You are a precise CV parser that extracts structured information. Always return valid JSON only."),
        HumanMessage(content=prompt)
    ]

    try:
        response = llm.invoke(messages)
        content = response.content.strip()

        # Remove markdown code blocks if present
        if content.startswith("```json"):
            content = content[7:]
        if content.startswith("```"):
            content = content[3:]
        if content.endswith("```"):
            content = content[:-3]
        content = content.strip()

        standardized_cv = json.loads(content)

        # Validate required fields
        required_fields = ["name", "headline", "city", "country", "skills", "experience", "education"]
        for field in required_fields:
            if field not in standardized_cv:
                if field == "skills":
                    standardized_cv[field] = []
                else:
                    standardized_cv[field] = None

        # Ensure skills is a list of strings
        if not isinstance(standardized_cv["skills"], list):
            standardized_cv["skills"] = []
        standardized_cv["skills"] = [str(s) for s in standardized_cv["skills"]]

        # Ensure experience and education are lists
        if not isinstance(standardized_cv["experience"], list):
            standardized_cv["experience"] = []
        if not isinstance(standardized_cv["education"], list):
            standardized_cv["education"] = []

        return standardized_cv

    except Exception as e:
        print(f"Error parsing CV: {e}")
        return {
            "name": None,
            "headline": None,
            "city": None,
            "country": None,
            "skills": [],
            "experience": [],
            "education": []
        }


def standardize_all_cvs(all_cvs: list, llm) -> list:
    """Convert all CV texts to standardized templates."""
    standardized_cvs = []

    for cv in all_cvs:
        print(f"Processing {cv['file']}...")
        standardized_cv = convert_cv_to_standardized_template(cv["text"], llm)
        standardized_cv["file"] = cv["file"]
        standardized_cvs.append(standardized_cv)
        print(f"✓ Completed {cv['file']}\n")

    return standardized_cvs


# Module 2: LinkedIn Search
def filter_by_headline_exact(candidates: list, cv_headline: str) -> list:
    """
    Phase 1: Filter by exact headline match (case-insensitive).
    Returns candidates with exact match OR empty headline.
    """
    if not cv_headline:
        return candidates

    filtered = []
    cv_headline_lower = cv_headline.lower()

    for candidate in candidates:
        candidate_headline = candidate.get("headline", "").strip()
        # Keep if: empty headline OR exact match
        if not candidate_headline or candidate_headline.lower() == cv_headline_lower:
            filtered.append(candidate)

    return filtered


async def filter_by_headline_llm(candidates: list, cv_headline: str, llm) -> list:
    """
    Phase 2: Filter by semantic headline similarity using LLM.
    Only called if exact match returns 0 results.
    """
    if not cv_headline or not candidates:
        return candidates

    # Prepare candidate info for LLM
    candidate_info = []
    for idx, candidate in enumerate(candidates):
        candidate_info.append({
            "index": idx,
            "id": candidate.get("id"),
            "name": candidate.get("name", ""),
            "headline": candidate.get("headline", "").strip() or "(empty)"
        })

    prompt = f"""You are a job title matching expert. Your task is to identify which LinkedIn candidates have a headline/job title that is semantically similar to the CV headline.

CV Headline: "{cv_headline}"

Candidates:
{json.dumps(candidate_info, indent=2, ensure_ascii=False)}

Instructions:
1. For each candidate, decide if their headline is semantically SIMILAR to the CV headline
2. Similar means: same profession, related roles, or overlapping job functions
   - Examples of SIMILAR: "Investment Professional" ↔ "Senior Investment Analyst"
   - Examples of NOT similar: "Investment Professional" ↔ "Software Engineer"
3. Always keep candidates with "(empty)" headline (index them)
4. Return ONLY a JSON array of candidate indices to KEEP

Example output format:
[0, 2, 5, 7]

Return the list of indices to keep:"""

    try:
        messages = [
            SystemMessage(content="You are a precise job title matching expert. Return only a valid JSON array of indices."),
            HumanMessage(content=prompt)
        ]

        response = llm.invoke(messages)
        content = response.content.strip()

        # Strip markdown fences
        if content.startswith("```json"):
            content = content[7:]
        if content.startswith("```"):
            content = content[3:]
        if content.endswith("```"):
            content = content[:-3]
        content = content.strip()

        keep_indices = json.loads(content)

        # Filter candidates by indices
        filtered = [candidates[i] for i in keep_indices if 0 <= i < len(candidates)]

        return filtered

    except Exception as e:
        print(f"  Error in LLM headline filtering: {e}")
        # Fallback: keep all candidates
        return candidates


async def rank_by_location_similarity(candidates: list, cv_city: str, cv_country: str, llm) -> list:
    """Rank candidates by location similarity using LLM."""
    if not candidates:
        return []

    if len(candidates) == 1:
        return [candidates[0]["id"]]

    candidate_info = []
    for candidate in candidates:
        location = candidate.get("location", "")
        parts = location.split(",")
        candidate_city = parts[0].strip() if len(parts) > 0 else ""
        candidate_country = parts[1].strip() if len(parts) > 1 else ""

        candidate_info.append({
            "id": candidate["id"],
            "name": candidate.get("name", ""),
            "headline": candidate.get("headline", ""),
            "city": candidate_city,
            "country": candidate_country,
            "location": location
        })

    prompt = f"""You are a location matching expert. Your task is to rank LinkedIn candidates by how well their location matches the CV location.

CV Location:
- City: {cv_city or "Not specified"}
- Country: {cv_country or "Not specified"}

Candidates to rank:
{json.dumps(candidate_info, indent=2, ensure_ascii=False)}

Instructions:
1. Compare each candidate's location (city and country) with the CV location
2. Consider exact matches, partial matches, and semantic similarity
3. Rank candidates from BEST match to WORST match
4. Return ONLY a JSON array of candidate IDs in ranked order (best match first)

Example output format:
[123, 456, 789]

Return the ranked list of IDs as JSON array:"""

    try:
        messages = [
            SystemMessage(content="You are a precise location matching expert. Return only valid JSON array of IDs."),
            HumanMessage(content=prompt)
        ]

        response = llm.invoke(messages)
        content = response.content.strip()

        if content.startswith("```json"):
            content = content[7:]
        if content.startswith("```"):
            content = content[3:]
        if content.endswith("```"):
            content = content[:-3]
        content = content.strip()

        ranked_ids = json.loads(content)

        candidate_ids = {c["id"] for c in candidates}
        ranked_ids = [id for id in ranked_ids if id in candidate_ids]

        for candidate in candidates:
            if candidate["id"] not in ranked_ids:
                ranked_ids.append(candidate["id"])

        return ranked_ids

    except Exception as e:
        print(f"Error ranking candidates: {e}")
        return [c["id"] for c in candidates]


async def search_linkedin_for_cv(standardized_cv: dict, mcp_tools: list, llm) -> list:
    """Search LinkedIn for a CV candidate and return ranked list of matching profile IDs."""
    search_tool = None
    for tool in mcp_tools:
        if tool.name == "search_linkedin_people":
            search_tool = tool
            break

    if not search_tool:
        print("Warning: search_linkedin_people tool not found")
        return []

    cv_name = standardized_cv.get("name", "")
    cv_headline = standardized_cv.get("headline", "")
    cv_city = standardized_cv.get("city", "")
    cv_country = standardized_cv.get("country", "")

    if not cv_name:
        print("Warning: CV has no name, cannot search LinkedIn")
        return []

    try:
        # Only search by name, no location filter
        search_args = {
            "q": cv_name,
            "limit": 20,
            "fuzzy": True
        }

        print(f"Searching LinkedIn for: {cv_name}")
        search_results = await search_tool.ainvoke(search_args)

        # Debug: print raw search results
        print(f"  DEBUG: search_results type = {type(search_results)}")
        print(f"  DEBUG: search_results = {search_results}")

        candidates = []
        if isinstance(search_results, list):
            for msg in search_results:
                if isinstance(msg, dict) and msg.get("type") == "text" and "text" in msg:
                    try:
                        parsed = json.loads(msg["text"])
                        if isinstance(parsed, list):
                            candidates.extend(parsed)
                        elif isinstance(parsed, dict) and "id" in parsed:
                            candidates.append(parsed)
                    except (json.JSONDecodeError, TypeError):
                        pass
        elif isinstance(search_results, dict) and "id" in search_results:
            candidates = [search_results]

        print(f"Found {len(candidates)} LinkedIn candidates")
        if candidates:
            print(f"  Sample candidate: {candidates[0]}")

        if not candidates:
            return []

        # Phase 1: Exact headline match
        filtered_candidates = filter_by_headline_exact(candidates, cv_headline)
        print(f"After exact headline filtering: {len(filtered_candidates)} candidates")

        # Phase 2: Check if we only have empty-headline candidates (no exact matches)
        # If so, use LLM for semantic matching on ALL candidates
        if filtered_candidates and cv_headline:
            has_exact_match = any(
                candidate.get("headline", "").strip().lower() == cv_headline.lower()
                for candidate in filtered_candidates
            )

            if not has_exact_match:
                # Only empty-headline candidates, need LLM semantic matching
                print(f"  Only empty-headline candidates found, trying LLM semantic matching...")
                filtered_candidates = await filter_by_headline_llm(candidates, cv_headline, llm)
                print(f"  After LLM headline filtering: {len(filtered_candidates)} candidates")
        elif not filtered_candidates and cv_headline:
            # No candidates at all (shouldn't happen with current logic, but keep as fallback)
            print(f"  No candidates, trying LLM semantic matching...")
            filtered_candidates = await filter_by_headline_llm(candidates, cv_headline, llm)
            print(f"  After LLM headline filtering: {len(filtered_candidates)} candidates")

        if not filtered_candidates:
            return []

        ranked_ids = await rank_by_location_similarity(
            filtered_candidates,
            cv_city,
            cv_country,
            llm
        )

        print(f"Ranked {len(ranked_ids)} candidate IDs")
        return ranked_ids

    except Exception as e:
        print(f"Error searching LinkedIn: {e}")
        return []


async def search_linkedin_for_all_cvs(standardized_cvs: list, mcp_tools: list, llm) -> list:
    """Search LinkedIn for all standardized CVs."""
    print("\n" + "=" * 80)
    print("MODULE 2: Searching LinkedIn for all CVs")
    print("=" * 80)

    linkedin_ids_list = []

    for idx, cv in enumerate(standardized_cvs):
        print(f"\n[{idx + 1}/{len(standardized_cvs)}] Processing {cv.get('file', 'Unknown')}")
        linkedin_ids = await search_linkedin_for_cv(cv, mcp_tools, llm)
        linkedin_ids_list.append(linkedin_ids)

        if linkedin_ids:
            print(f"✓ Found {len(linkedin_ids)} matching LinkedIn profile(s): {linkedin_ids}")
        else:
            print("✗ No matching LinkedIn profiles found")

        if idx < len(standardized_cvs) - 1:
            await asyncio.sleep(1.0)

    return linkedin_ids_list


# Module 3: Fetch LinkedIn profiles
def convert_linkedin_to_standardized(raw_profile: dict) -> dict:
    """Convert a raw LinkedIn profile to standardized format."""
    skills = []
    for s in raw_profile.get("skills", []):
        if isinstance(s, dict):
            skills.append(s.get("name", ""))
        elif isinstance(s, str):
            skills.append(s)

    experience = []
    for exp in raw_profile.get("experience", []):
        experience.append({
            "company": exp.get("company"),
            "title": exp.get("title"),
            "seniority": exp.get("seniority"),
            "start_year": exp.get("start_year"),
            "end_year": exp.get("end_year"),
        })

    education = []
    for edu in raw_profile.get("education", []):
        education.append({
            "school": edu.get("school"),
            "degree": edu.get("degree"),
            "field": edu.get("field"),
            "end_year": edu.get("end_year"),
        })

    return {
        "name": raw_profile.get("name"),
        "headline": raw_profile.get("headline"),
        "city": raw_profile.get("city"),
        "country": raw_profile.get("country"),
        "skills": skills,
        "experience": experience,
        "education": education,
    }


async def fetch_linkedin_profiles(linkedin_ids: list, mcp_tools: list) -> list:
    """Fetch LinkedIn profiles for a list of IDs and convert to standardized format."""
    profile_tool = None
    for tool in mcp_tools:
        if tool.name == "get_linkedin_profile":
            profile_tool = tool
            break

    if not profile_tool:
        print("Warning: get_linkedin_profile tool not found")
        return []

    profiles = []
    for idx, person_id in enumerate(linkedin_ids):
        try:
            raw = await profile_tool.ainvoke({"person_id": person_id})

            parsed = raw
            if isinstance(raw, list):
                for msg in raw:
                    if isinstance(msg, dict) and msg.get("type") == "text" and "text" in msg:
                        try:
                            parsed = json.loads(msg["text"])
                        except (json.JSONDecodeError, TypeError):
                            pass

            if isinstance(parsed, dict) and "error" not in parsed:
                standardized = convert_linkedin_to_standardized(parsed)
                standardized["linkedin_id"] = person_id
                profiles.append(standardized)
            else:
                print(f"  Warning: profile {person_id} returned error or invalid data")

        except Exception as e:
            print(f"  Error fetching profile {person_id}: {e}")

        if idx < len(linkedin_ids) - 1:
            await asyncio.sleep(1.0)

    return profiles


async def fetch_all_linkedin_profiles(linkedin_ids_list: list, mcp_tools: list, standardized_cvs: list) -> list:
    """Fetch LinkedIn profiles for every CV."""
    print("\n" + "=" * 80)
    print("MODULE 3: Fetching LinkedIn profiles")
    print("=" * 80)

    all_profiles = []

    for idx, ids in enumerate(linkedin_ids_list):
        cv_label = standardized_cvs[idx].get("file", f"CV_{idx+1}")
        if not ids:
            print(f"\n[{idx+1}/{len(linkedin_ids_list)}] {cv_label}: No IDs → skipping")
            all_profiles.append(None)
            continue

        print(f"\n[{idx+1}/{len(linkedin_ids_list)}] {cv_label}: Fetching {len(ids)} profile(s)...")
        profiles = await fetch_linkedin_profiles(ids, mcp_tools)
        all_profiles.append(profiles if profiles else None)

        if profiles:
            for p in profiles:
                print(f"  ✓ {p.get('name')} — {p.get('headline')}")
        else:
            print("  ✗ Could not retrieve any valid profiles")

        if idx < len(linkedin_ids_list) - 1:
            await asyncio.sleep(1.0)

    return all_profiles


# Module 4: Scoring
def build_comparison_pairs(cv: dict, li: dict):
    """Build comparison pairs for LLM batch comparison."""
    pairs = []

    pairs.append({
        "cv_val": str(cv.get("city") or ""),
        "li_val": str(li.get("city") or ""),
        "category": "city",
        "sub_field": "city",
        "index": 0,
    })

    pairs.append({
        "cv_val": str(cv.get("country") or ""),
        "li_val": str(li.get("country") or ""),
        "category": "country",
        "sub_field": "country",
        "index": 0,
    })

    cv_skills = cv.get("skills", []) or []
    li_skills = li.get("skills", []) or []
    li_skills_str = ", ".join(li_skills) if li_skills else "(none)"
    for i, skill in enumerate(cv_skills):
        pairs.append({
            "cv_val": str(skill),
            "li_val": li_skills_str,
            "category": "skills",
            "sub_field": "skill_item",
            "index": i,
        })

    cv_exps = cv.get("experience", []) or []
    li_exps = li.get("experience", []) or []
    for i, cv_exp in enumerate(cv_exps):
        li_exps_summary = json.dumps(li_exps, ensure_ascii=False) if li_exps else "(none)"
        for field in ["company", "title", "seniority", "start_year", "end_year"]:
            pairs.append({
                "cv_val": str(cv_exp.get(field) or ""),
                "li_val": li_exps_summary,
                "category": "experience",
                "sub_field": field,
                "index": i,
            })

    cv_edus = cv.get("education", []) or []
    li_edus = li.get("education", []) or []
    for i, cv_edu in enumerate(cv_edus):
        li_edus_summary = json.dumps(li_edus, ensure_ascii=False) if li_edus else "(none)"
        for field in ["school", "degree", "field", "end_year"]:
            pairs.append({
                "cv_val": str(cv_edu.get(field) or ""),
                "li_val": li_edus_summary,
                "category": "education",
                "sub_field": field,
                "index": i,
            })

    num_cv_skills = len(cv_skills) if cv_skills else 1
    num_exp_entries = len(cv_exps) if cv_exps else 0
    num_edu_entries = len(cv_edus) if cv_edus else 0

    weight_info = {
        "city_max": 1,
        "country_max": 1,
        "skills_max": 1,
        "skills_count": num_cv_skills,
        "experience_max": num_exp_entries * 5,
        "education_max": num_edu_entries * 4,
    }

    return pairs, weight_info


def python_string_compare(pairs: list) -> list:
    """
    Phase 1: Fast Python string comparison (no LLM).
    Compare strings after lowercasing and stripping whitespace.
    Returns list of 1 (match) or 0 (no match).
    """
    verdicts = []

    for pair in pairs:
        cv_val = str(pair["cv_val"]).lower().strip()
        li_val = str(pair["li_val"]).lower().strip()

        # Skip empty values
        if not cv_val or cv_val == "none" or not li_val or li_val == "(none)":
            verdicts.append(0)
            continue

        category = pair["category"]

        # For simple fields: exact match
        if category in ["city", "country"]:
            verdicts.append(1 if cv_val == li_val else 0)

        # For skills: check if CV skill is in LinkedIn skills list
        elif category == "skills":
            # li_val is comma-separated list of skills
            verdicts.append(1 if cv_val in li_val else 0)

        # For experience/education: check if CV value appears in LinkedIn JSON
        elif category in ["experience", "education"]:
            # li_val is JSON string containing all entries
            verdicts.append(1 if cv_val in li_val else 0)

        else:
            verdicts.append(0)

    return verdicts


def batch_llm_compare(pairs: list, llm) -> list:
    """
    Phase 2: LLM semantic comparison (only called if Phase 1 score <= 0.5).
    Ask LLM to judge each pair as MATCH (1) or NO_MATCH (0).
    """
    if not pairs:
        return []

    table_lines = []
    for idx, p in enumerate(pairs):
        table_lines.append(
            f'{idx}. category={p["category"]}, field={p["sub_field"]}, '
            f'CV_value="{p["cv_val"]}", LinkedIn_value="{p["li_val"]}"'
        )
    table_text = "\n".join(table_lines)

    prompt = f"""You are a strict data comparison judge for CV verification.

For each numbered pair below, decide if the CV value is CONFIRMED by the LinkedIn value.

RULES:
- Return 1 if the CV value is present / equivalent in the LinkedIn data
- Return 0 if the CV value is NOT found or clearly contradicts the LinkedIn data
- For skills: return 1 if the specific CV skill appears anywhere in the LinkedIn skills list
- For experience/education fields: return 1 if the specific field value can be found in ANY of the LinkedIn entries
- If either value is empty/null/"None", return 0

PAIRS:
{table_text}

Return ONLY a JSON array of integers (1 or 0), one per pair, in the same order.
Example for 3 pairs: [1, 0, 1]

JSON array:"""

    try:
        messages = [
            SystemMessage(content="You are a precise data comparison judge. Return only a valid JSON array of 1s and 0s."),
            HumanMessage(content=prompt),
        ]
        response = llm.invoke(messages)
        content = response.content.strip()

        if content.startswith("```json"):
            content = content[7:]
        if content.startswith("```"):
            content = content[3:]
        if content.endswith("```"):
            content = content[:-3]
        content = content.strip()

        verdicts = json.loads(content)

        if len(verdicts) != len(pairs):
            print(f"  Warning: LLM returned {len(verdicts)} verdicts but expected {len(pairs)}, padding with 0")
            while len(verdicts) < len(pairs):
                verdicts.append(0)
            verdicts = verdicts[: len(pairs)]

        return [int(v) for v in verdicts]

    except Exception as e:
        print(f"  Error in LLM comparison: {e}")
        return [0] * len(pairs)


def compute_score(pairs: list, verdicts: list, weight_info: dict) -> float:
    """Compute final score from verdicts and weight info."""
    earned = {
        "city": 0.0,
        "country": 0.0,
        "skills": 0.0,
        "experience": 0.0,
        "education": 0.0,
    }

    skills_count = weight_info["skills_count"]

    for pair, v in zip(pairs, verdicts):
        cat = pair["category"]
        if cat == "city":
            earned["city"] += v
        elif cat == "country":
            earned["country"] += v
        elif cat == "skills":
            earned["skills"] += v / skills_count
        elif cat == "experience":
            earned["experience"] += v
        elif cat == "education":
            earned["education"] += v

    total_max = (
        weight_info["city_max"]
        + weight_info["country_max"]
        + weight_info["skills_max"]
        + weight_info["experience_max"]
        + weight_info["education_max"]
    )

    if total_max == 0:
        return 0.0

    total_earned = sum(earned.values())
    final = total_earned / total_max

    print(f"    city:       {earned['city']:.2f} / {weight_info['city_max']}")
    print(f"    country:    {earned['country']:.2f} / {weight_info['country_max']}")
    print(f"    skills:     {earned['skills']:.2f} / {weight_info['skills_max']}")
    print(f"    experience: {earned['experience']:.0f} / {weight_info['experience_max']}")
    print(f"    education:  {earned['education']:.0f} / {weight_info['education_max']}")
    print(f"    TOTAL:      {total_earned:.2f} / {total_max}  →  {final:.4f}")

    return final


def score_cv_against_profiles(cv: dict, profiles: list, llm):
    """
    Score one CV against its ranked LinkedIn profiles.
    Two-phase strategy:
    1. Fast Python string matching (no LLM)
    2. LLM semantic matching (only if Phase 1 score <= 0.5)
    """
    import time

    if not profiles:
        return None

    # ========== Phase 1: Fast Python String Matching ==========
    print(f"  Phase 1: Fast Python string matching (no LLM)...")
    best_score_phase1 = 0.0

    for rank, li_profile in enumerate(profiles):
        print(f"    Profile #{rank+1} (id={li_profile.get('linkedin_id')})", end=" ")
        pairs, weight_info = build_comparison_pairs(cv, li_profile)
        verdicts = python_string_compare(pairs)  # No LLM!
        score = compute_score(pairs, verdicts, weight_info)

        if score > best_score_phase1:
            best_score_phase1 = score

        if score > 0.5:
            print(f"\n  → Phase 1 score {score:.4f} > 0.5, accepting without LLM! ✅")
            return score

    print(f"\n  → Phase 1 best score: {best_score_phase1:.4f} (≤ 0.5)")

    # ========== Phase 2: LLM Semantic Matching ==========
    print(f"  Phase 2: LLM semantic matching (with 5s delays)...")
    best_score_phase2 = 0.0

    for rank, li_profile in enumerate(profiles):
        print(f"  Comparing with LinkedIn profile #{rank+1} (id={li_profile.get('linkedin_id')})")
        pairs, weight_info = build_comparison_pairs(cv, li_profile)
        verdicts = batch_llm_compare(pairs, llm)  # Use LLM
        score = compute_score(pairs, verdicts, weight_info)

        if score > best_score_phase2:
            best_score_phase2 = score

        if score > 0.5:
            print(f"  → Phase 2 score {score:.4f} > 0.5, accepting this profile")
            return score

        # Add longer delay between LLM calls to avoid rate limit
        if rank < len(profiles) - 1:
            print(f"  → Waiting 5s before next LLM comparison...")
            time.sleep(5.0)

    print(f"  → Phase 2 best score: {best_score_phase2:.4f}")
    return best_score_phase2


# Module 5: Facebook fallback
async def facebook_find_alternative_names(cv_name: str, mcp_tools: list) -> list:
    """Search Facebook for alternative names."""
    fb_search = None
    fb_profile = None
    for tool in mcp_tools:
        if tool.name == "search_facebook_users":
            fb_search = tool
        elif tool.name == "get_facebook_profile":
            fb_profile = tool

    if not fb_search or not fb_profile:
        print("  Warning: Facebook tools not found")
        return []

    try:
        search_results = await fb_search.ainvoke({
            "q": cv_name,
            "limit": 20,
            "fuzzy": True,
        })

        fb_users = []
        if isinstance(search_results, list):
            for msg in search_results:
                if isinstance(msg, dict) and msg.get("type") == "text" and "text" in msg:
                    try:
                        parsed = json.loads(msg["text"])
                        if isinstance(parsed, list):
                            fb_users.extend(parsed)
                        elif isinstance(parsed, dict) and "id" in parsed:
                            fb_users.append(parsed)
                    except (json.JSONDecodeError, TypeError):
                        pass

        print(f"  Facebook search for '{cv_name}': {len(fb_users)} result(s)")

        if not fb_users:
            return []

    except Exception as e:
        print(f"  Error searching Facebook: {e}")
        return []

    alternative_names = set()
    cv_name_lower = cv_name.strip().lower()

    for idx, user in enumerate(fb_users):
        fb_id = user.get("id")
        display_name = user.get("display_name", "")

        print(f"\n  [FB User {idx+1}] ID: {fb_id}")
        print(f"    display_name: '{display_name}'")

        # Collect display_name if different from CV name
        if display_name.strip().lower() != cv_name_lower and display_name.strip():
            alternative_names.add(display_name.strip())
            print(f"    ✓ Added display_name: '{display_name.strip()}'")

        try:
            raw = await fb_profile.ainvoke({"user_id": fb_id})

            parsed = raw
            if isinstance(raw, list):
                for msg in raw:
                    if isinstance(msg, dict) and msg.get("type") == "text" and "text" in msg:
                        try:
                            parsed = json.loads(msg["text"])
                        except (json.JSONDecodeError, TypeError):
                            pass

            if isinstance(parsed, dict):
                original_name = parsed.get("original_name", "")
                print(f"    original_name: '{original_name}'")

                # Collect original_name if different from CV name
                if original_name and original_name.strip().lower() != cv_name_lower:
                    # Check if it's the same as display_name (to avoid duplicates)
                    if original_name.strip() != display_name.strip():
                        alternative_names.add(original_name.strip())
                        print(f"    ✓ Added original_name: '{original_name.strip()}'")
                    else:
                        print(f"    ⊙ original_name == display_name (already added)")

                # Also check display_name from profile (in case it differs from search result)
                dn = parsed.get("display_name", "")
                if dn and dn.strip().lower() != cv_name_lower and dn.strip() != display_name.strip():
                    alternative_names.add(dn.strip())
                    print(f"    ✓ Added profile display_name: '{dn.strip()}'")

        except Exception as e:
            print(f"    ✗ Error fetching FB profile {fb_id}: {e}")

        if idx < len(fb_users) - 1:
            await asyncio.sleep(1.0)

    alt_list = list(alternative_names)
    print(f"\n  📋 Summary:")
    print(f"    CV name: '{cv_name}'")
    if alt_list:
        print(f"    Alternative names found ({len(alt_list)}): {alt_list}")
    else:
        print("    No alternative names found")

    return alt_list


async def module5_facebook_fallback(
    standardized_cvs: list,
    scores_list: list,
    mcp_tools: list,
    llm,
) -> list:
    """For every CV whose score is None, try Facebook fallback."""
    print("\n" + "=" * 80)
    print("MODULE 5: Facebook fallback for unscored CVs")
    print("=" * 80)

    for idx, (cv, score) in enumerate(zip(standardized_cvs, scores_list)):
        label = cv.get("file", f"CV_{idx+1}")

        # Skip if score > 0.5 (already well-verified)
        if score is not None and score > 0.5:
            print(f"\n[{idx+1}] {label}: score {score:.4f} > 0.5 → skip")
            continue

        # Try Facebook fallback for: None or score ≤ 0.5
        if score is None:
            print(f"\n[{idx+1}] {label}: score is None → trying Facebook fallback")
        else:
            print(f"\n[{idx+1}] {label}: score {score:.4f} ≤ 0.5 → trying Facebook fallback")

        cv_name = cv.get("name", "")
        if not cv_name:
            print("  No name in CV → cannot search Facebook")
            continue

        alt_names = await facebook_find_alternative_names(cv_name, mcp_tools)

        if not alt_names:
            print("  No alternative names → score stays None")
            continue

        best_score = 0.0

        for alt_name in alt_names:
            print(f"\n  Trying alternative name: '{alt_name}'")

            cv_alt = dict(cv)
            cv_alt["name"] = alt_name

            li_ids = await search_linkedin_for_cv(cv_alt, mcp_tools, llm)
            if not li_ids:
                print(f"    No LinkedIn matches for '{alt_name}'")
                continue

            li_profiles = await fetch_linkedin_profiles(li_ids, mcp_tools)
            if not li_profiles:
                print(f"    Could not fetch profiles for '{alt_name}'")
                continue

            s = score_cv_against_profiles(cv, li_profiles, llm)
            if s is not None and s > best_score:
                best_score = s
                print(f"    New best score: {best_score:.4f}")

            if best_score > 0.5:
                print(f"    Score > 0.5 → accepting")
                break

        if best_score > 0.0:
            scores_list[idx] = best_score
            print(f"  Final score for {label}: {best_score:.4f}")
        else:
            print(f"  No valid score obtained → score stays None")

    return scores_list


########################################################
# Combine modules into agent
########################################################


async def run_cv_verification():
    """Main async function that wraps all the CV verification logic."""

    # ========== Connect to MCP Server ==========
    print("\n" + "=" * 80)
    print("CONNECTING TO MCP SERVER")
    print("=" * 80)

    client = MultiServerMCPClient({
        "social_graph": {
            "transport": "http",
            "url": "https://ftec5660.ngrok.app/mcp",
            "headers": {"ngrok-skip-browser-warning": "true"}
        }
    })

    print("Fetching MCP tools...")
    mcp_tools = await client.get_tools()
    print(f"✓ Connected! Loaded {len(mcp_tools)} tools: {[t.name for t in mcp_tools]}")

    # ========== Initialize LLM ==========
    print("\n" + "=" * 80)
    print("INITIALIZING LLM")
    print("=" * 80)
    print("Model: gemini-2.0-flash")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=GEMINI_VERTEX_API_KEY,
        temperature=0,
    )
    print("✓ LLM initialized successfully")

    # ========== Module 1: Standardize CVs ==========
    print("\n" + "=" * 80)
    print("MODULE 1: Converting CVs to standardized template")
    print("=" * 80)

    standardized_cvs = standardize_all_cvs(all_cvs, llm)
    print(f"\n✓ Module 1 complete: {len(standardized_cvs)} CVs standardized")

    for cv in standardized_cvs:
        print(f"\n📄 {cv['file']}")
        print(json.dumps(cv, indent=2, ensure_ascii=False))
        print("\n" + "-" * 80 + "\n")

    # ========== Module 2: LinkedIn Search ==========
    linkedin_ids_list = await search_linkedin_for_all_cvs(standardized_cvs, mcp_tools, llm)

    print("\n" + "=" * 80)
    print("MODULE 2 RESULTS: LinkedIn Profile IDs")
    print("=" * 80)
    for idx, (cv, ids) in enumerate(zip(standardized_cvs, linkedin_ids_list)):
        print(f"\n{cv.get('file', 'Unknown')}: {ids if ids else 'No matches'}")

    # ========== Module 3: Fetch LinkedIn Profiles ==========
    linkedin_profiles_list = await fetch_all_linkedin_profiles(linkedin_ids_list, mcp_tools, standardized_cvs)

    print("\n" + "=" * 80)
    print("MODULE 3 RESULTS: Standardized LinkedIn Profiles")
    print("=" * 80)
    for idx, (cv, profiles) in enumerate(zip(standardized_cvs, linkedin_profiles_list)):
        label = cv.get("file", f"CV_{idx+1}")
        if profiles is None:
            print(f"\n{label}: No LinkedIn data")
        else:
            print(f"\n{label}: {len(profiles)} profile(s)")
            for p in profiles:
                print(json.dumps(p, indent=2, ensure_ascii=False))

    # ========== Module 4: Scoring ==========
    print("\n" + "=" * 80)
    print("MODULE 4: Scoring CVs against LinkedIn profiles")
    print("=" * 80)

    scores_list = []

    for idx, (cv, profiles) in enumerate(zip(standardized_cvs, linkedin_profiles_list)):
        label = cv.get("file", f"CV_{idx+1}")
        print(f"\n[{idx+1}/{len(standardized_cvs)}] {label}")

        if profiles is None:
            print("  No LinkedIn profiles available → score = None")
            scores_list.append(None)
        else:
            score = score_cv_against_profiles(cv, profiles, llm)
            scores_list.append(score)

    print("\n" + "=" * 80)
    print("MODULE 4 RESULTS: Scores")
    print("=" * 80)
    for idx, (cv, s) in enumerate(zip(standardized_cvs, scores_list)):
        label = cv.get("file", f"CV_{idx+1}")
        print(f"  {label}: {s if s is not None else 'None (no LinkedIn data)'}")

    # ========== Module 5: Facebook Fallback ==========
    scores_list = await module5_facebook_fallback(standardized_cvs, scores_list, mcp_tools, llm)
    print(f"\n✓ Module 5 complete")

    # ========== Final Output ==========
    scores = [s if s is not None else 0.0 for s in scores_list]

    print("\n" + "=" * 80)
    print("FINAL SCORES (Ready for evaluation)")
    print("=" * 80)
    for idx, (cv, s) in enumerate(zip(standardized_cvs, scores)):
        label = cv.get("file", f"CV_{idx+1}")
        print(f"  {label}: {s:.4f}")

    return scores, standardized_cvs

########################################################
# Generates a verification report
########################################################

def generate_verification_report(standardized_cvs: list, scores: list, threshold: float = 0.5):
    """
    Generate a detailed verification report showing results for each CV.

    Args:
        standardized_cvs: List of standardized CV dictionaries
        scores: List of verification scores (0-1)
        threshold: Decision threshold (default 0.5)
    """
    print("\n" + "=" * 80)
    print("📊 CV VERIFICATION REPORT")
    print("=" * 80)
    print(f"Threshold: {threshold}")
    print(f"Total CVs: {len(scores)}")
    print("=" * 80)

    # Summary stats
    verified = sum(1 for s in scores if s > threshold)
    rejected = len(scores) - verified

    print(f"\n✅ Verified: {verified}")
    print(f"❌ Rejected: {rejected}")
    print("\n" + "-" * 80)

    # Detailed results for each CV
    print("\n📋 DETAILED RESULTS:\n")

    for idx, (cv, score) in enumerate(zip(standardized_cvs, scores)):
        cv_file = cv.get("file", f"CV_{idx+1}")
        cv_name = cv.get("name", "Unknown")
        decision = "✅ VERIFIED" if score > threshold else "❌ REJECTED"

        print(f"[{idx+1}] {cv_file}")
        print(f"    Name:     {cv_name}")
        print(f"    Score:    {score:.4f}")
        print(f"    Decision: {decision}")

        # Classification
        if score > 0.8:
            confidence = "🟢 High Confidence"
        elif score > 0.5:
            confidence = "🟡 Medium Confidence"
        elif score > 0.3:
            confidence = "🟠 Low Match"
        else:
            confidence = "🔴 Very Poor Match"

        print(f"    Confidence: {confidence}")
        print()

    print("=" * 80)
    print("END OF REPORT")
    print("=" * 80)



```
# This is formatted as code
```

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [40]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [41]:
scores, standardized_cvs = await run_cv_verification() # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)

# Generate verification report
generate_verification_report(standardized_cvs, scores, threshold=0.5)



CONNECTING TO MCP SERVER
Fetching MCP tools...
✓ Connected! Loaded 6 tools: ['search_facebook_users', 'get_facebook_profile', 'get_facebook_mutual_friends', 'search_linkedin_people', 'get_linkedin_profile', 'get_linkedin_interactions']

INITIALIZING LLM
Model: gemini-2.0-flash
✓ LLM initialized successfully

MODULE 1: Converting CVs to standardized template
Processing CV_1.pdf...
✓ Completed CV_1.pdf

Processing CV_2.pdf...
✓ Completed CV_2.pdf

Processing CV_3.pdf...
✓ Completed CV_3.pdf

Processing CV_4.pdf...
✓ Completed CV_4.pdf

Processing CV_5.pdf...
✓ Completed CV_5.pdf


✓ Module 1 complete: 5 CVs standardized

📄 CV_1.pdf
{
  "name": "John Smith",
  "headline": "Marketing Professional",
  "city": "Singapore",
  "country": "Singapore",
  "skills": [
    "Content Creation",
    "SEO",
    "Social Media"
  ],
  "experience": [
    {
      "company": "ByteDance",
      "title": "Engineer",
      "seniority": "mid",
      "start_year": 2020,
      "end_year": null
    }
  ],
  "edu